**About** : This notebook is used to validate models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

In [ ]:
%matplotlib inline

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import timm
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *
from collections import Counter
from numerize.numerize import numerize

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from util.plots import *
from inference_main import kfold_inference

### Inference

In [ ]:
EXP_FOLDER = "../logs/2023-07-23/6/"  # v2-s stride 1 single frame

In [ ]:
df = pd.DataFrame({"img_path": glob.glob('../output/goes16_false_color_2/*.png')})

In [ ]:
df['record_id'] = df['img_path'].apply(lambda x: x.split('/')[-1][:-4])

In [ ]:
df['mask_path'] = df['img_path']
df['folder'] = ""
df['has_contrail'] = ""

In [ ]:
preds = kfold_inference(df, EXP_FOLDER, use_fp16=True, save=False, batch_size=4)

In [ ]:
masks = preds[0][:, 0]

In [ ]:
PLOT = False
SAVE = True

m = 10

In [ ]:
SAVE_FOLDER = "../output/goes16_pl/"

os.makedirs(SAVE_FOLDER, exist_ok=True)
os.makedirs(SAVE_FOLDER + "images/", exist_ok=True)
os.makedirs(SAVE_FOLDER + "masks/", exist_ok=True)

In [ ]:
for i in tqdm(range(len(df))):
    img = cv2.imread(df['img_path'][i])
    
    probs = masks[i][:img.shape[0], :img.shape[1]]
    mask = (probs > 0.5).astype(int)
    
    if mask.sum() < 100:  # Not enough pixels
        continue

    # Crop
    x0 = np.argmax(((mask > 0).sum(1) > 5))
    x1 = mask.shape[0] - np.argmax(((mask > 0).sum(1) > 5)[::-1])
    y0 = np.argmax(((mask > 0).sum(0) > 5))
    y1 = mask.shape[1] - np.argmax(((mask > 0).sum(0) > 5)[::-1])

    x0 = max(x0 - m, 0)
    y0 = max(y0 - m, 0)
    x1 = min(x1 + m, mask.shape[0])
    y1 = min(y1 + m, mask.shape[1])
    
    img_f = img[x0: x1, y0: y1]
    probs_f = probs[x0: x1, y0: y1]
    mask_f = mask[x0: x1, y0: y1]

    if PLOT:  #  or not (i % 400):
        hw = min(img_f.shape[0], img_f.shape[1])
        plot_sample(img_f[:hw, :hw], mask_f[:hw, :hw, None], n=2, figsize=(20, 10))
        
    if img_f.shape[0] < 256:
        continue
    elif img_f.shape[1] < 256:
        continue
    
    if SAVE:
        np.save(SAVE_FOLDER + "masks/" + df['record_id'][i] + ".npy", probs_f)
        cv2.imwrite(SAVE_FOLDER + "images/" + df['record_id'][i] + ".png", img_f)
    
#     break

In [ ]:
for i in tqdm(range(len(df))):
    
    try:
        mask = np.load(SAVE_FOLDER + "masks/" + df['record_id'][i] + ".npy")
        img = cv2.imread(SAVE_FOLDER + "images/" + df['record_id'][i] + ".png")
    except:
        continue
    
    assert img.shape[:2] == mask.shape

In [ ]:
df = pd.DataFrame({
    "img_path": sorted(glob.glob(SAVE_FOLDER + 'images/*.png')),
    "mask_path": sorted(glob.glob(SAVE_FOLDER + 'masks/*.npy')),
})

df['record_id'] = df['img_path'].apply(lambda x: x.split('/')[-1][:-4])
df['folder'] = ""
df['has_contrail'] = True

# df.to_csv('../output/df_goes16_may.csv', index=False)

### Results

In [ ]:
from data.dataset import ContrailDataset
from data.transforms import get_transfos

In [ ]:
dataset = ContrailDataset(df, transforms=get_transfos(crop=True))

In [ ]:
img, mask, _ = dataset[0]

In [ ]:
for idx in np.random.choice(len(dataset), 3):
    image, mask, y = dataset[idx]

    if len(image.size()) == 3:
        plot_sample(image.numpy().transpose(1, 2, 0), mask[:1].numpy().transpose(1, 2, 0) > 0.5)
    else:
        plot_sample(image[0].numpy().transpose(1, 2, 0).copy(), mask[:1].numpy().transpose(1, 2, 0) > 0.5)
#         plot_sample(image[4].numpy().transpose(1, 2, 0).copy(), mask[:1].numpy().transpose(1, 2, 0))

    break

Done ! 